<a href="https://colab.research.google.com/github/DenisMihailapov/open_stt/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Анализ аудиоданных с помощью глубокого обучения и Python

In [ ]:
import random, os, csv, splitfolders, pathlib 
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from numpy import argmax

import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image

import librosa
import librosa.display
import IPython.display

# Предварительная обработка sklearn
from sklearn.model_selection import train_test_split

# Keras
import keras
from keras import layers
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from keras.models import Sequential
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
PATH = f'/content/drive/My Drive/ASR/'

###Готовим данные из файлов wav (папка genres)


In [ ]:
#Создаём спектрограммы
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(PATH+f'Audio/genres/{g}').mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(PATH+f'Audio/genres/{g}'):
        songname = PATH+f'Audio/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        print(y.shape)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(PATH+f'Audio/spectrograms/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

In [ ]:
#Разбиваем на обучение и тест. (80/20)
splitfolders.ratio(PATH+f'Audio/spectrograms/', output=PATH+f'Audio/data/', seed=1337, ratio=(.8, .2)) # значения по умолчанию


Copying files: 0 files [00:00, ? files/s]
Copying files: 2 files [00:00,  6.85 files/s]
Copying files: 3 files [00:00,  4.32 files/s]
Copying files: 4 files [00:00,  4.20 files/s]
Copying files: 5 files [00:01,  3.61 files/s]
Copying files: 6 files [00:01,  3.51 files/s]
Copying files: 7 files [00:02,  3.13 files/s]
Copying files: 8 files [00:02,  3.31 files/s]
Copying files: 9 files [00:02,  3.66 files/s]
Copying files: 10 files [00:02,  3.61 files/s]
Copying files: 11 files [00:03,  3.25 files/s]
Copying files: 12 files [00:03,  2.31 files/s]
Copying files: 13 files [00:04,  2.44 files/s]
Copying files: 14 files [00:04,  2.47 files/s]
Copying files: 15 files [00:05,  2.45 files/s]
Copying files: 16 files [00:05,  2.14 files/s]
Copying files: 17 files [00:05,  2.45 files/s]
Copying files: 18 files [00:06,  1.95 files/s]
Copying files: 19 files [00:07,  1.73 files/s]
Copying files: 20 files [00:07,  1.74 files/s]
Copying files: 21 files [00:08,  1.62 files/s]
Copying files: 22 files [

In [ ]:
#Генераторы для новых данных
train_datagen = ImageDataGenerator(
        rescale=1./255,       # изменение масштаба всех значений пикселей с 0 до 255, 
                              # после этого шага они будут находится в диапазоне (0,1)
        shear_range=0.2,      # применение случайных преобразований
        zoom_range=0.2,       # увеличение масштаба
        horizontal_flip=True) # горизонтальный поворот

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:

training_set = train_datagen.flow_from_directory(
        PATH+f'Audio/data/train', target_size=(64, 64),
        batch_size=32,  class_mode='categorical',
        shuffle = False)
test_set = test_datagen.flow_from_directory(
        PATH+f'Audio/data/val',  target_size=(64, 64),
        batch_size=32, class_mode='categorical',
        shuffle = False )

Found 805 images belonging to 10 classes.
Found 205 images belonging to 10 classes.


###Создаем сверточную нейронную сеть

In [ ]:
model = Sequential()

# первый скрытый слой
model.add(Conv2D(32, (3, 3), strides=(2, 2), input_shape=(64, 64, 3)))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))

# второй скрытый слой
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))

# третий скрытый слой
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))

# слой выравнивания
model.add(Flatten())
model.add(Dropout(rate=0.5))

# полносвязный слой
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

# выходной слой
model.add(Dense(10))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 31, 31, 32)        896       
_________________________________________________________________
average_pooling2d (AveragePo (None, 15, 15, 32)        0         
_________________________________________________________________
activation (Activation)      (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
average_pooling2d_1 (Average (None, 7, 7, 64)          0         
_________________________________________________________________
activation_1 (Activation)    (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          3

In [ ]:
#Обучение нейронки
epochs = 200
batch_size = 8
learning_rate = 0.01
decay_rate = learning_rate / epochs
momentum = 0.9

sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
print(test_set)

In [ ]:

model.fit_generator(training_set, steps_per_epoch=len(training_set), 
        epochs=50, validation_data=test_set, validation_steps=200)

Epoch 1/50
26/26 [==============================] - 7s 280ms/step - loss: 2.0318 - accuracy: 0.1714 - val_loss: 1.8917 - val_accuracy: 0.1951
Epoch 2/50
26/26 [==============================] - 6s 249ms/step - loss: 2.0156 - accuracy: 0.1814
Epoch 3/50
26/26 [==============================] - 6s 233ms/step - loss: 1.9729 - accuracy: 0.1925
Epoch 4/50
26/26 [==============================] - 6s 229ms/step - loss: 1.9622 - accuracy: 0.1739
Epoch 5/50
26/26 [==============================] - 6s 227ms/step - loss: 1.9525 - accuracy: 0.1665
Epoch 6/50
26/26 [==============================] - 6s 231ms/step - loss: 1.9256 - accuracy: 0.1714
Epoch 7/50
26/26 [==============================] - 6s 237ms/step - loss: 1.9348 - accuracy: 0.1665
Epoch 8/50
26/26 [==============================] - 6s 228ms/step - loss: 1.9112 - accuracy: 0.1752
Epoch 9/50
26/26 [==============================] - 6s 225ms/step - loss: 1.8957 - accuracy: 0.1478
Epoch 10/50
26/26 [==============================] - 6s 22

In [ ]:
# Оценка модели
model.evaluate_generator(generator=test_set, steps=50)

[1.6306549310684204, 0.35121950507164]